<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/portfolios/three_assets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import uniform

def random_wts(num_assets) :
    w = uniform.rvs(0,1,num_assets)
    return w/w.sum()

# start with 3,000 random portfolios
ports = [random_wts(3) for i in range(3000)]

# now add portfolios on the boundaries since
# the random portfolios tend to be in the interior
ports1 = [(0,x,1-x) for x in np.linspace(0,1,21)]
ports2 = [(x,0,1-x) for x in np.linspace(0,1,21)]
ports3 = [(x,1-x,0) for x in np.linspace(0,1,21)]
ports = ports+ports1+ports2+ports3


def data(mn1,mn2,mn3,sd1,sd2,sd3,c12,c13,c23) :

  # put means and stdevs in arrays
  mns = pd.Series(np.array([mn1,mn2,mn3]), index=['stock1','stock2','stock3'])
  sds = np.array([sd1,sd2,sd3])

  # compute the correlation matrix
  C = np.diag([1.0,1.0,1.0])
  C[0,1] = c12/100
  C[0,2] = c13/100
  C[1,0] = c12/100
  C[1,2] = c23/100
  C[2,0] = c13/100
  C[2,1] = c23/100

  # compute the covariance matrix
  D = np.diag(sds/100)
  C = D @ C @ D

  # calculate the means and stdevs of all of the portfolios
  df = pd.DataFrame(dtype=float,index=range(len(ports)),\
                    columns=['mean','stdev','wt1','wt2','wt3'])
  df['mean'] = [p @ mns/100 for p in ports]
  df['stdev'] = [np.sqrt(p @ C @ p) for p in ports]

  # add the weights of the portfolios
  df['wt1'] = [p[0] for p in ports]
  df['wt2'] = [p[1] for p in ports]
  df['wt3'] = [p[2] for p in ports]
  return df

# example
mn1 = 0.1
mn2 = 0.06
mn3 = 0.12
sd1 = 0.15
sd2 = 0.1
sd3 = 0.2
c12 = 0.3
c13 = 0.3
c23 = 0.3
df = data(mn1,mn2,mn3,sd1,sd2,sd3,c12,c13,c23)
df

,mean,stdev,wt1,wt2,wt3
0,0.000840,0.000785,0.297116,0.501777,0.201107
1,0.000900,0.000893,0.218754,0.426368,0.354878
2,0.001114,0.001437,0.245709,0.061145,0.693146
3,0.001033,0.001129,0.637818,0.065988,0.296194
4,0.000958,0.000951,0.328260,0.293935,0.377805
...,...,...,...,...,...
3058,0.000920,0.001217,0.800000,0.200000,0.000000
3059,0.000940,0.001284,0.850000,0.150000,0.000000
3060,0.000960,0.001354,0.900000,0.100000,0.000000
3061,0.000980,0.001426,0.950000,0.050000,0.000000
